# Jayansh

In [174]:
import requests as req
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as bs

In [175]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
}

In [176]:
url = []
url.append("https://www.espncricinfo.com/series/indian-premier-league-2012-520932/match-schedule-fixtures-and-results")
url.append("https://www.espncricinfo.com/series/indian-premier-league-2013-586733/match-schedule-fixtures-and-results")
url.append("https://www.espncricinfo.com/series/pepsi-indian-premier-league-2014-695871/match-schedule-fixtures-and-results")
url.append("https://www.espncricinfo.com/series/pepsi-indian-premier-league-2015-791129/match-schedule-fixtures-and-results")

In [177]:
matches = []
base_url="https://www.espncricinfo.com/"
for u in url:
    res = req.get(u, headers=headers)
    soup = bs(res.text,'html.parser')
    soup.prettify()
    atags = soup.find_all("a", class_ ="ds-no-tap-higlight")
    for atag in atags:
        link = atag.get("href")
        if link and "full-scorecard"in link and "indian-premier-league"in link:
            matches.append(base_url+link)
matches

['https://www.espncricinfo.com//series/indian-premier-league-2012-520932/chennai-super-kings-vs-mumbai-indians-1st-match-548306/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/kolkata-knight-riders-vs-delhi-daredevils-2nd-match-548307/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/mumbai-indians-vs-pune-warriors-3rd-match-548308/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/rajasthan-royals-vs-kings-xi-punjab-4th-match-548309/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/royal-challengers-bangalore-vs-delhi-daredevils-5th-match-548310/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/deccan-chargers-vs-chennai-super-kings-6th-match-548311/full-scorecard',
 'https://www.espncricinfo.com//series/indian-premier-league-2012-520932/rajasthan-royals-vs-kolkata-knight-riders-7th-matc

In [178]:
len(matches)

272

In [179]:
match_data = {
        'Match Type':[],
        'POTM': [],
        'Venue': [],
        'Team1': [],
        'Team2': [],
        'Toss_Winner':[],
        'Toss_Decision': [],
        'Winner': [],
        'Win_Margin':[],
        'Result_Type': [],
        'Target_Runs': [],
        'Match':[],
        'Season':[]
}

In [180]:
match_types=["1st Qualifying","2nd Qualifying", "Elimination", "Final","Qualifier 1", "Qualifier 2", "Eliminator"]
temp=[]
l=1
for match in matches:
    res = req.get(match, headers=headers)
    soup = bs(res.text, 'html.parser')
    soup.prettify()
    data = soup.find_all("div", class_ = "ds-text-tight-m ds-font-regular ds-text-typo-mid3")
    for x in data:
        if match_types[0] in x.text or match_types[4] in x.text:
            match_data["Match Type"].append(match_types[4])
        elif match_types[1] in x.text or match_types[5] in x.text:
            match_data["Match Type"].append(match_types[5])
        elif match_types[2] in x.text or match_types[6] in x.text:
            match_data["Match Type"].append(match_types[6])
        elif match_types[3] in x.text:
            match_data["Match Type"].append(match_types[3])
        else:
            match_data["Match Type"].append("League")
    
    potm = soup.find("span", class_ = "ds-text-tight-m ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block ds-cursor-pointer")
    if potm:
        match_data["POTM"].append(potm.text)
    else:
        match_data["POTM"].append("Nil")
    
    tables = soup.find("table", class_ = "ds-w-full ds-table ds-table-sm ds-table-auto")
    for table in tables:
        venue_tag = table.find("a", class_ = "ds-inline-flex ds-items-start ds-leading-none")
        venue = venue_tag.get("title")
        match_data["Venue"].append(venue)
        toss = table.find("span", class_="ds-text-tight-s ds-font-regular")
        if "," in toss.text:
            toss_win = toss.text.split(',')[0]
            toss_dec = toss.text.split(',')[1]
            match_data["Toss_Winner"].append(toss_win)
            if "field" in toss_dec:
                match_data["Toss_Decision"].append("Field first")
            else:
                match_data["Toss_Decision"].append("Bat first")
        else:
            match_data["Toss_Winner"].append("Nil")
            match_data["Toss_Decision"].append("Nil")
        season = soup.find_all("span", class_ ="ds-text-tight-s ds-font-medium ds-text-typo ds-underline ds-decoration-ui-stroke hover:ds-text-typo-primary hover:ds-decoration-ui-stroke-primary ds-block")[2]
        match_data["Season"].append(season.text)
        
    teams = soup.find_all("span", class_="ds-text-tight-l ds-font-bold ds-text-typo hover:ds-text-typo-primary ds-block ds-truncate")
    match_data["Team1"].append(teams[0].text)
    match_data["Team2"].append(teams[1].text)
    ind  = soup.find("strong", class_="ds-uppercase ds-text-tight-m")
    if "RESULT" in ind:
        margin_line = soup.find("p", class_="ds-text-tight-s ds-font-medium ds-truncate ds-text-typo")
        if "tied" in margin_line.text:
            match_data["Win_Margin"].append("Tied")
            match_data["Winner"].append("Tied")
        else:
            margin = margin_line.text.split("by")[1]
            if '(' in margin:
                match_data["Win_Margin"].append(margin.split('(')[0])
            else:
                match_data["Win_Margin"].append(margin)

            winner = margin_line.text.split("won")[0]
            match_data["Winner"].append(winner)
    else:
        match_data["Win_Margin"].append("Nil")
        match_data["Winner"].append("Nil")

    target_div = soup.find("div", class_ = "ds-text-compact-m ds-text-typo ds-text-right ds-whitespace-nowrap")
    if target_div:
        target = target_div.find("strong")
        if target:
            if '/' in target.text:
                match_data["Target_Runs"].append(int(target.text.split('/')[0]) + 1)
            else:
                match_data["Target_Runs"].append(int(target.text) + 1)
        else:
            match_data["Target_Runs"].append("Nil")
    else:
        match_data["Target_Runs"].append("Nil")

x=1
for i in range(271):
    if match_data["Season"][i]==match_data["Season"][i+1]:
        match_data["Match"].append(x)
        x+=1
    else:
        match_data["Match"].append(x)
        x=1

match_data["Match"].append(x)

for m in match_data["Win_Margin"]:
    if "wickets" in m:
        match_data["Result_Type"].append("Wickets")
    elif "runs" in m:
        match_data["Result_Type"].append("Runs")
    else:
        match_data["Result_Type"].append(m)



In [195]:
print(len(match_data["Match Type"]))
print(len(match_data["POTM"]))
print(len(match_data["Venue"]))
print(len(match_data["Team1"]))
print(len(match_data["Team2"]))
print(len(match_data["Toss_Winner"]))
print(len(match_data["Toss_Decision"]))
print(len(match_data["Winner"]))
print(len(match_data["Win_Margin"]))
print(len(match_data["Result_Type"]))
print(len(match_data["Target_Runs"]))
print(len(match_data["Match"]))
print(len(match_data["Season"]))

272
272
272
272
272
272
272
272
272
272
272
272
272


In [197]:
# match_data["Match Type"].clear()
# match_data["POTM"].clear()
# match_data["Venue"].clear()
# match_data["Team1"].clear()
# match_data["Team2"].clear()
# match_data["Toss_Winner"].clear()
# match_data["Toss_Decision"].clear()
# match_data["Winner"].clear()
# match_data["Win_Margin"].clear()
# match_data["Result_Type"].clear()
# match_data["Target_Runs"].clear()
# match_data["Match"].clear()
# match_data["Season"].clear()

In [199]:
pd.DataFrame(match_data)

,Match Type,POTM,Venue,Team1,Team2,Toss_Winner,Toss_Decision,Winner,Win_Margin,Result_Type,Target_Runs,Match,Season
0,League,Richard Levi,"MA Chidambaram Stadium, Chepauk, Chennai",Chennai Super Kings,Mumbai Indians,Mumbai Indians,Field first,MI,8 wickets,Wickets,113,1,2012
1,League,Irfan Pathan,"Eden Gardens, Kolkata",Kolkata Knight Riders,Delhi Daredevils,Delhi Daredevils,Field first,Daredevils,8 wickets,Wickets,98,2,2012
2,League,Steven Smith,"Wankhede Stadium, Mumbai",Pune Warriors,Mumbai Indians,Mumbai Indians,Field first,Warriors,28 runs,Runs,130,3,2012
3,League,Ajinkya Rahane,"Sawai Mansingh Stadium, Jaipur",Rajasthan Royals,Kings XI Punjab,Kings XI Punjab,Field first,RR,31 runs,Runs,192,4,2012
4,League,AB de Villiers,"M Chinnaswamy Stadium, Bangalore",Royal Challengers Bangalore,Delhi Daredevils,Delhi Daredevils,Field first,RCB,20 runs,Runs,158,5,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,League,Mitchell McClenaghan,"Rajiv Gandhi International Stadium, Uppal, Hyd...",Sunrisers Hyderabad,Mumbai Indians,Sunrisers Hyderabad,Bat first,MI,9 wickets,Wickets,114,56,2015
268,Qualifier 1,Kieron Pollard,"Wankhede Stadium, Mumbai",Mumbai Indians,Chennai Super Kings,Mumbai Indians,Bat first,MI,25 runs,Runs,188,57,2015
269,Eliminator,AB de Villiers,"Maharashtra Cricket Association Stadium, Pune",Royal Challengers Bangalore,Rajasthan Royals,Royal Challengers Bangalore,Bat first,RCB,71 runs,Runs,181,58,2015
270,Qualifier 2,Ashish Nehra,"JSCA International Stadium Complex, Ranchi",Royal Challengers Bangalore,Chennai Super Kings,Chennai Super Kings,Field first,CSK,3 wickets,Wickets,140,59,2015


In [201]:
df = pd.DataFrame(match_data)

In [205]:
df.to_csv('2012-2015.csv', sep='\t', index=False,header=True)